# 🚀 cuOpt Examples Launcher

Welcome to the cuOpt Examples Repository! This notebook helps you:
- 📥 Clone or update the latest cuopt-examples repository
- 🌿 Select and switch between different branches
- 📂 Interactively browse all available notebooks
- 🔗 Launch notebooks directly in Google Colab

---


## Step 1: Setup and Repository Management


In [ ]:
# Install required packages
%pip install ipywidgets gitpython -q


In [ ]:
import os
import sys
from pathlib import Path
import subprocess
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, Output, Dropdown, Layout, HTML as HTMLWidget
import json

# Configuration
REPO_URL = "https://github.com/NVIDIA/cuopt-examples.git"
DEFAULT_BRANCH = "branch-25.10"
REPO_NAME = "cuopt-examples"

# Detect if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    BASE_DIR = Path("/content")
else:
    BASE_DIR = Path.cwd()

REPO_PATH = BASE_DIR / REPO_NAME

print(f"Base directory: {BASE_DIR}")
print(f"Repository path: {REPO_PATH}")


## Step 2: Clone or Update Repository


In [ ]:
def run_command(command, cwd=None):
    """Execute a shell command and return output"""
    try:
        result = subprocess.run(
            command, 
            shell=True, 
            capture_output=True, 
            text=True, 
            cwd=cwd
        )
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

def clone_or_update_repo():
    """Clone repository if it doesn't exist, otherwise update it"""
    output = widgets.Output()
    
    with output:
        if REPO_PATH.exists():
            print(f"📂 Repository already exists at {REPO_PATH}")
            print("🔄 Fetching latest changes...")
            success, stdout, stderr = run_command("git fetch --all", cwd=REPO_PATH)
            if success:
                print("✅ Repository updated successfully!")
            else:
                print(f"⚠️  Warning: {stderr}")
        else:
            print(f"📥 Cloning repository from {REPO_URL}...")
            success, stdout, stderr = run_command(f"git clone {REPO_URL} {REPO_PATH}")
            if success:
                print("✅ Repository cloned successfully!")
            else:
                print(f"❌ Error: {stderr}")
                return
        
        # Show current branch
        success, current_branch, _ = run_command("git branch --show-current", cwd=REPO_PATH)
        if success:
            print(f"\n🌿 Current branch: {current_branch.strip()}")
    
    display(output)

# Run the clone/update
clone_or_update_repo()
